In [1]:
import numpy as np
import functools
import pandas as pd
import os
import copy
import operator

In [2]:
#df = pd.read_csv('./data/partition/ForecastDataforTesting_201712_6.csv')
#df = df.groupby(['xid', 'yid', 'date_id', 'hour']).mean()
#df = df.reset_index()
#df = df.drop('model', axis=1)
#df_city = pd.read_csv('data/CityData.csv')
#df['hour'] = df['hour']*60-180

In [3]:
df = pd.read_csv('./In_situMeasurementforTraining_201712_1.csv')
df['hour'] = df['hour']*60-180
df_city = pd.read_csv('./CityData.csv')

In [11]:
def find_neighbors(x, limit_x=[1, 548], limit_y=[1, 421]):
    xx = [(x[0]+i_i, x[1]+i_j) for i_i in [-1, 0, 1] for i_j in [-1, 0, 1] if abs(i_i)+abs(i_j)==1]
    
    xx = list(filter(lambda i: i[0]>=limit_x[0] and i[0]<=limit_x[1] and i[1]>=limit_y[0] and i[1]<=limit_y[1], xx))
    return xx

In [12]:
def pointwise_compare(x):
    x2 = np.concatenate([[np.nan], x[:-1]])
    return x!=x2

In [14]:
def _reconcile(s1, s2):
    pts = np.unique(np.sort(np.concatenate([s1._x, s2._x])))
    # Handle case when endpoints are inf
    cpts = pts.copy()
    cpts[0] = min(np.min(cpts[1:]), 0.) - 1
    cpts[-1] = max(np.max(cpts[:-1]), 0.) + 1
    mps = (cpts[1:] + cpts[:-1]) / 2.
    return [(pts, s(mps)) for s in (s1, s2)]


def _same_support(s1, s2):
    return np.all(s1._x[[0, -1]] == s2._x[[0, -1]])


def require_compatible(f):
    @functools.wraps(f)
    def wrapper(self, other, *args, **kwargs):
        if isinstance(other, StepFunction) and not _same_support(self, other):
            raise TypeError("Step functions have different support: %s vs. %s" % (
                self._x[[0, -1]], other._x[[0, -1]]))
        return f(self, other, *args, **kwargs)
    return wrapper


class StepFunction:
    '''A step function.'''

    def __init__(self, x, y):
        '''Initialize step function with breakpoints x and function values y.
        x and y are arrays such that
            f(z) = y[k], x[k] <= z < x[k + 1], 0 <= k < K.
        Thus, len(x) == len(y) + 1 and domain of f is (x[0], x[K]).
        '''
        if len(x) != 1 + len(y):
            raise RuntimeError("len(x) != 1 + len(y)")
        self._x = np.array(x)
        self._y = np.array(y, dtype='float')
        self._compress()

    @property
    def K(self):
        '''The number of steps.'''
        return len(self._y)

    def _compress(self):
        # Combine steps which have equal values
     #   ny = np.concatenate([[np.nan], self._y, [np.nan]])
     #   ys = np.diff(ny) != 0
        ys = np.concatenate([pointwise_compare(self._y), [True]])
        self._x = self._x[ys]
        self._y = self._y[ys[:-1]]

    def _binary_op(self, other, op, desc):
        if isinstance(other, StepFunction):
            (s1_x, s1_y), (s2_x, s2_y) = _reconcile(self, other)
            return StepFunction(s1_x, op(s1_y, s2_y))
        # Fall back to normal semantics otherwise
        return StepFunction(self._x, op(self._y, other))

    def __add__(self, other):
        return self._binary_op(other, operator.add, "add")

    def __radd__(self, other):
        return self + other

    def __sub__(self, other):
        return self._binary_op(other, operator.sub, "subtract")

    def __rsub__(self, other):
        return -self + other

    def __mul__(self, other):
        return self._binary_op(other, operator.mul, "multiply")

    def __rmul__(self, other):
        return self * other

    def __div__(self, other):
        return self._binary_op(other, operator.div, "divide")

    def __rdiv__(self, other):
        return (self ** -1) * other

    # Unary operations

    def __neg__(self):
        return StepFunction(self._x, -self._y)

    def __pow__(self, p):
        return StepFunction(self._x, pow(self._y, p))

    def __abs__(self):
        return StepFunction(self._x, abs(self._y))

    # Equality and comparison operators

    @require_compatible
    def __eq__(self, other):
        return (np.array_equal(self._x, other._x) and 
                np.array_equal(self._y, other._y))

    @require_compatible
    def __lt__(self, other):
        diff = other - self
        return np.all(diff._y > 0)

    @require_compatible
    def __le__(self, other):
        diff = other - self
        return np.all(diff._y >= 0)

    @require_compatible
    def __gt__(self, other):
        return -self < -other

    @require_compatible
    def __ge__(self, other):
        return -self <= -other

    def __call__(self, s):
        return self._y[np.searchsorted(self._x, s, side="right") - 1]

    def __repr__(self):
        return "StepFunction(x=%s, y=%s)" % (repr(self._x), repr(self._y))

    def integral(self):
        nz = self._y != 0
        d = np.diff(self._x)
        return (d[nz] * self._y[nz]).sum()
    
    def find_interval_value(self, x_interval):
        # [a, b)
     
        y = np.arange(np.argwhere(self._x<=x_interval[0])[-1], np.argwhere(self._x<x_interval[1])[-1]+1)
        yy = self._y[y]
        yyy = np.sort(np.unique(np.append(self._x[y], x_interval)))
        yyy = yyy[np.all(np.array([yyy>=x_interval[0], yyy<=x_interval[1]]), axis=0)]
        return np.array([yyy, yy])
   
    def extend_domain(self, new_value):
 
        if new_value < self._x[0]:
            self._x = np.insert(self._x, 0, new_value)
            self._y = np.insert(self._y, 0, float("Inf"))
            self.__init__(self._x, self._y)
        elif new_value > self._x[0]:
            self._x = np.append(self._x, new_value)
            self._y = np.append(self._y, float("Inf"))
            #self.__init__(self._x, self._y)
        else:
            pass
            

In [15]:
def hvdistance(x, y):
    return np.sum(np.abs(np.array(x) - np.array(y)))

In [16]:
class vertex_i:
    def __init__(self, vi, vs, td, ta):
        self.vi = vi
        self.Ti = np.array([hvdistance(vi, vs)*2, ta])
        self.Si = np.array([ta, ta])
        self.TAUi = None
        self.negihbor_Sij = {}
        self.ta = ta
        self.td = td
        
    #def update_Si(self):
        
    def update_TAUi_Si(self, interval_x):
        f = StepFunction(self.gi_d, self.gi_v)
        y = f.find_interval_value(interval_x)
        self.TAUi = y[1].min()
        # self.TAUi2 = y[1].min() + hvdistance(self.vi, ve)
        # self.Si = np.array([y[0][np.argmin(y[1])], self.ta])

        self.Si = np.array([min(self.gi_d[np.append(self.gi_v == self.TAUi, [False])]), self.ta])
    
    
    def create_negihbor(self, ta):
        
        x = find_neighbors(self.vi, limit_x=[1, 548], limit_y=[1, 421])
        y = [[ta-2, ta-2]]*len(x)
        
        self.negihbor_Sij = dict(zip(x, y))
    
    def create_gi(self):
        self.gi_d = np.array([self.td, self.ta])
        self.gi_v = np.array([float('Inf')])
    
    def update_gi(self, i_x, i_y):
          
        i_x = np.array(i_x)
        i_y = np.array(i_y)
       
        f1 = StepFunction(self.gi_d, self.gi_v)
        
        f2 = StepFunction(i_x, i_y)
        
        if 0 not in i_x:
            f2.extend_domain(self.td)
        if 1080 not in i_x:
            f2.extend_domain(self.ta)
        
        x1 = np.sort(np.unique(np.append(self.gi_d, i_x)))
        y1 = [min(f1(i), f2(i)) if i in i_x else f1(i) for i in x1[:-1]]
        
        f = StepFunction(x1, y1)
        
        self.gi_d = f._x
        self.gi_v = f._y
    
        

In [22]:
def compute_minimum_cost(v_s, v_e, t_d, t_a, df):

    Q = {}
    QQ = []
    Q[v_s] = vertex_i(v_s, v_s, t_d, t_a)
    Q[v_s].gi_v = np.array([0])
    Q[v_s].gi_d = np.array([t_d, t_a])
    t = Q[v_s].Ti
    Q[v_s].update_TAUi_Si([t_d, t_a])
    Q[v_s].create_negihbor(t_a)
    
    
    v_i = v_s
    run_times = 1
    while v_i != v_e:
        
        for i in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            # if (Q[v_i].Si[0] < (t_a - 2)) and (Q[v_i].TAUi != float('Inf')):
            if (Q[v_i].Si[0] < (t_a - 2)):
                R_ij = [Q[v_i].Si[0], t_a-2]
                 
               
                x = [R_ij[0], Q[v_i].negihbor_Sij[i][0]]
                
                
                domain_ij = [x[0]+2, x[1]+2]
        
            
              
                if domain_ij[0] < domain_ij[1]:
                    f_ij = df[(df['xid'] == i[0]) & (df['yid'] == i[1])]
                    f_ij.sort_values(by='hour', ascending=True, inplace=True)
                
                #f_ij_d = np.append(f_ij.hour, t_a)
                #f_ij_v = np.where(f_ij.wind>=15, 1440, 0) 
                
                    f_ij = StepFunction(np.append(f_ij.hour, t_a), np.where(f_ij.wind>=15, 1440, 0))
                    
                    
                    f_ij = f_ij.find_interval_value(domain_ij)
                
                    update_d = f_ij[0]
                    update_v = f_ij[1] + Q[v_i].TAUi
                
                    Q[v_i].negihbor_Sij[i] = R_ij
                
                    if i not in [iii for iii in Q]:
                        Q[i] = vertex_i(i, v_s, t_d, t_a)
                        Q[i].create_gi()
                        Q[i].create_negihbor(t_a)
                    
                    domain_j = [Q[i].Ti[0], Q[i].Si[0]]
                    
                
                    
                    if Q[i].Ti[0] < Q[i].Si[0]:
                        Q[i].update_gi(update_d, update_v)

                
                        run_times += 1
                        print run_times
                
                        Q[i].update_TAUi_Si(domain_j)
                
 
                    if i not in [iii for iii, jjj in QQ]:
                        QQ.append((i, Q[i].TAUi))
                              
                
                
        if (Q[v_i].Ti[0] < Q[v_i].Si[0]):
            
            domain_i = [Q[v_i].Ti[0], Q[v_i].Si[0]]
            
            Q[v_i].update_TAUi_Si(domain_i)
            print domain_i
            print Q[v_i].gi_d
            print Q[v_i].gi_v
            print '----------'
            
            QQ.append((v_i, Q[v_i].TAUi))
            
        
        QQ = sorted(QQ, key=operator.itemgetter(1), reverse=False)
        
        v_i = QQ.pop(0)[0]
    return Q

In [23]:

p_result = compute_minimum_cost((50, 50), (60, 60), 0, 1080, df)

/home/k1758472/.conda/envs/DS_Py27/lib/python2.7/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [24]:
print p_result[(60, 60)].Ti
print p_result[(60, 60)].Si

[  40 1080]
[  40 1080]


In [25]:
print p_result[(60, 60)].gi_d
print p_result[(60, 60)].gi_v

[   0   40  240  360  420  480 1080]
[   inf     0.  1440.     0.  1440.     0.]


In [26]:
print p_result[(50, 51)].Ti
print p_result[(50, 51)].Si
print p_result[(50, 51)].gi_d
print p_result[(50, 51)].gi_v

[   2 1080]
[   2 1080]
[   0    2 1080]
[ inf   0.]


In [32]:
print p_result[(50, 55)].Ti
print p_result[(50, 55)].Si
print p_result[(50, 55)].gi_d
print p_result[(50, 55)].gi_v

[  10 1080]
[  10 1080]
[   0   10 1080]
[ inf   0.]


In [29]:
print p_result[(60, 60)].gi_d
print p_result[(60, 60)].gi_v

[   0   40  240  360  420  480 1080]
[   inf     0.  1440.     0.  1440.     0.]


In [33]:
print p_result[(50, 51)].ta

1080


In [34]:
print p_result[(50, 60)].Ti
print p_result[(50, 60)].Si
print p_result[(50, 60)].gi_d
print p_result[(50, 60)].gi_v

[  20 1080]
[  20 1080]
[   0   20 1080]
[ inf   0.]


In [35]:
print p_result[(60, 60)].Ti
print p_result[(60, 60)].Si
print p_result[(60, 60)].gi_d
print p_result[(60, 60)].gi_v

[  40 1080]
[  40 1080]
[   0   40  240  360  420  480 1080]
[   inf     0.  1440.     0.  1440.     0.]


In [36]:
print p_result[(59, 60)].Ti
print p_result[(59, 60)].Si
print p_result[(59, 60)].gi_d
print p_result[(59, 60)].gi_v

[  38 1080]
[  38 1080]
[   0   38  420  480 1080]
[   inf     0.  1440.     0.]


In [30]:
def path_selection(g, v_s, v_e, df):
    
    v_i = v_e
    path = []
    waiting_time = []

    while v_i != v_s:
        
        check_break = 0
        
        stop_find = 0
        
        gi = StepFunction(g[v_i].gi_d, g[v_i].gi_v)
            
        ti = g[v_i].gi_d[np.argmin(g[v_i].gi_v)]
        f_ji = StepFunction(np.append(f_ji.hour, g[v_i].ta), np.where(f_ji.wind>=15, 1440, 0))
        
        
        for j in find_neighbors(v_i, limit_x=[1, 548], limit_y=[1, 421]):
            print '---------'
            print j
            check_break = 1
            

            
            if (j in g) and (stop_find == 0):
               
                tj = g[j].gi_d[np.argmin(g[j].gi_v)]
                gj = StepFunction(g[j].gi_d, g[j].gi_v)
                f_ji = df[(df['xid'] == v_i[0]) & (df['yid'] == v_i[1])]
                f_ji.sort_values(by='hour', ascending=True, inplace=True)
                
                
                
                
     
                
                find_idx_j = (g[j].gi_v + f_ji(ti-2)) == gi(ti)
                
                find_idx_j = np.append(find_idx_j, [False])
              

                if sum(find_idx_j) > 0:
                    
                    find_idx_j = np.argmin(g[j].gi_d[find_idx_j])
                    tj = g[j].gi_d[find_idx_j]

                    if tj <= (ti-2):
                
                        path.append(j)
                        waiting_time.append(ti-2-tj)
                    
                        v_i = j
                        
                        check_break = 0
                        stop_find = 1
                        print check_break
                
            else:
                continue
        if check_break !=0:
            break
            
    path.reverse()
    waiting_time.reverse()
    return {'path': path, 'wating_time': waiting_time}    

In [31]:
path_selection(p_result, (50, 50), (55, 55), df)

---------
(54, 55)
0
---------
(55, 54)
---------
(55, 56)
---------
(56, 55)


/home/k1758472/.conda/envs/DS_Py27/lib/python2.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


{'path': [(54, 55)], 'wating_time': [18]}

In [125]:
print p_result[(60, 59)].gi_d
print p_result[(60, 59)].gi_v
print p_result[(60, 59)].Si
print p_result[(60, 59)].Ti

[    0.    98.  1080.]
[ inf   0.]
[   38.  1080.]
[  38 1080]


In [91]:
p_result[(60, 59)].update_TAUi_Si([38, 1080])

In [92]:
print p_result[(60, 59)].gi_d
print p_result[(60, 59)].gi_v
print p_result[(60, 59)].Si
print p_result[(60, 59)].Ti

[    0.    98.  1080.]
[ inf   0.]
[   98.  1080.]
[  38 1080]


In [115]:
sum(np.array([True, True, False]))

2